In [0]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn import linear_model, preprocessing
import os
import glob
import pandas as pd
from pandas import ExcelWriter
from pandas import ExcelFile
import nltk
from nltk import tokenize
from nltk.stem import WordNetLemmatizer
import re 
# nltk.download('stopwords')
# nltk.download()
from nltk.corpus import stopwords 

In [2]:
nltk.download('punkt')
nltk.download('all')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping corpora/biocreative_ppi.zip.
[nltk_data]    | Downloading package brown to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/brown.zip.
[nltk_data]    | Downloading package brown_tei to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/brown_tei.zip.
[nltk_data]    | Downloading package cess_cat to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/cess_cat.zip.
[nltk_data]    | Downloading package cess_esp to /root/nltk_data...
[nltk_data]    |   Unzipping corpora

True

In [0]:
# data_path='./'
# os.chdir(data_path)
files=glob.glob('*.txt')

sents=[]
tagsP=[]
tagsF=[]
emptyin=[]
for f in files:
    file = open(f)
    df= pd.read_csv(file,delimiter='\t',encoding='utf-8')
    emptyin=df[df['Past'].isnull()].index.tolist()
    st=np.delete(np.array(df['sentence'].values),emptyin)
    ft=np.delete(np.array(df['Future'].values),emptyin)
    pt=np.delete(np.array(df['Past'].values),emptyin)
    sents.extend(list(st))
    tagsF.extend(list(ft))
    tagsP.extend(list(pt))
    print(emptyin)
X=np.asarray(sents)
yf=np.asarray(tagsF)
yp=np.asarray(tagsP)
print(len(X))
print(len(yf))
print(len(yp))

[103, 266, 280, 301, 314, 392, 412, 474, 664, 685, 688, 778, 875, 926, 975]
[49, 82, 195, 258, 533, 541, 577, 648, 726, 942]
[51, 90, 214, 273, 304, 404, 458, 511, 527, 640, 664, 769, 911, 995]
2961
2961
2961


In [0]:
f='projectivity.txt'
file=open(f)
df= pd.read_csv(file,delimiter='\t',encoding='utf-8')
emptyin=df[df['Past'].isnull()].index
dt=df.loc[:, 'id':'Future']
dt=dt.drop(emptyin)
print(len(dt))
# dt=np.delete(np.array(df.loc[:, 'A':'E'].values),emptyin)



2961


In [0]:
def split_data_and_store(df):
  train, validation, test = np.split(df, [int(.6*len(df)), int(.8*len(df))])
  train.to_csv('train', sep='\t')
  validation.to_csv('validation', sep='\t')
  test.to_csv('test', sep='\t')
  print('Done!')
  

In [0]:
split_data_and_store(dt)

In [0]:
def load_train_val(train,val):
  df= pd.read_csv(train,delimiter='\t',encoding='utf-8')
  tr_st=np.array(df['sentence'].values)
  tr_ft=np.array(df['Future'].values)
  tr_pt=np.array(df['Past'].values)
  df= pd.read_csv(val,delimiter='\t',encoding='utf-8')
  val_st=np.array(df['sentence'].values)
  val_ft=np.array(df['Future'].values)
  val_pt=np.array(df['Past'].values)
  return tr_st,tr_ft,tr_pt, val_st,val_ft,val_pt


In [0]:
def load_test(test):
  df= pd.read_csv(test,delimiter='\t',encoding='utf-8')
  ts_st=np.array(df['sentence'].values)
  ts_ft=np.array(df['Future'].values)
  ts_pt=np.array(df['Past'].values)
  return ts_st,ts_ft,ts_pt

In [0]:
def preprocess_sentences(X):
  documents = []
  stemmer = WordNetLemmatizer()
  for sen in range(0, len(X)):
    document = re.sub(r'\W', ' ', str(X[sen]))
    document = re.sub(r'\s+[a-zA-Z]\s+', ' ', document)
    document = re.sub(r'^"', '', document)
    document = re.sub(r'"$', '', document)
    document = re.sub(r'\^[a-zA-Z]\s+', ' ', document) 
    document = re.sub(r'\s+', ' ', document, flags=re.I)
    document = re.sub(r'^b\s+', '', document)
    document = document.lower()
    #     document = document.split()
    #     document = [stemmer.lemmatize(word) for word in document]
    #     document = ' '.join(document)
    documents.append(document)
  print(len(documents))
  return documents

In [0]:
def sent_to_features(documents):
  feature_docs=[]
  for document in documents:
    tokens=nltk.word_tokenize(document)
    bigrams = list(nltk.bigrams(tokens))
    #pos_bigrams = list(nltk.bigrams([pos for (word,pos) in nltk.pos_tag(tokens)]))
    document=[word+' '+pos for (word,pos) in nltk.pos_tag(tokens)]
    document.extend([w1+'_'+w2 for (w1,w2) in bigrams])
    #     document.extend([w1+'_'+w2 for (w1,w2) in pos_bigrams])
    #     document.extend([pos for (word,pos) in nltk.pos_tag(tokens)])
    #     print(document)
    #     document = document.split()
    #     document = [stemmer.lemmatize(word) for word in document]
    document = ' '.join(document)
    feature_docs.append(document)
  return feature_docs

In [0]:
# documents = []
# stemmer = WordNetLemmatizer()
# for sen in range(0, len(X)):
#   document = re.sub(r'\W', ' ', str(X[sen]))
#   document = re.sub(r'\s+[a-zA-Z]\s+', ' ', document)
#   document = re.sub(r'^"', '', document)
#   document = re.sub(r'"$', '', document)
#   document = re.sub(r'\^[a-zA-Z]\s+', ' ', document) 
#   document = re.sub(r'\s+', ' ', document, flags=re.I)
#   document = re.sub(r'^b\s+', '', document)
#   document = document.lower()
#   #     document = document.split()
#   #     document = [stemmer.lemmatize(word) for word in document]
#   #     document = ' '.join(document)
#   documents.append(document)
# print(len(documents))

986


In [7]:
#loading train and validation data
trainfile='train.txt'
valfile='validation.txt'
tr_st,tr_ft,tr_pt, val_st,val_ft,val_pt=load_train_val(trainfile,valfile)

#preprocessing
tr_st = preprocess_sentences(tr_st)
val_st = preprocess_sentences(val_st)

#feature extraction
tr_st = sent_to_features(tr_st)
val_st = sent_to_features(val_st)

print('Train size: ',len(tr_st))
print('Val size: ',len(val_st))

1776
592
Train size:  1776
Val size:  592


In [8]:
print(tr_st[1])

friends NNS and CC adversaries NNS abroad RB were VBD asking VBG whether IN america NN had VBD lost VBN its PRP$ nerve NN friends_and and_adversaries adversaries_abroad abroad_were were_asking asking_whether whether_america america_had had_lost lost_its its_nerve


In [0]:
# print(stopwords.words('english'))

In [0]:
from sklearn.feature_extraction.text import CountVectorizer  
vectorizer = CountVectorizer()  #, stop_words=stopwords.words('english')
documents = np.concatenate((tr_st,val_st))
print(len(documents))
vectorizer.fit(documents)


2368


CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)

In [0]:
tr_st = vectorizer.transform(tr_st)
val_st = vectorizer.transform(val_st)

In [0]:
print(tr_st.shape)
# print(X[0][:1000])

(1776, 39471)


In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer  
tfidfconverter = TfidfVectorizer()#min_df=2, max_df=0.8, stop_words=stopwords.words('english')
documents = np.concatenate((tr_st,val_st))
print(len(documents))
tfidfconverter.fit(documents) 
tr_st = tfidfconverter.transform(tr_st)
val_st = tfidfconverter.transform(val_st)

2368


In [10]:
print(tr_st.shape)
# print(Xt[0][:1000])


(1776, 39471)


In [0]:
# from sklearn.model_selection import train_test_split  
# Xtr, Xts, ytr, yts = train_test_split(X, yf, test_size=0.3, random_state=42)

In [0]:
from sklearn import svm

svc = svm.SVC(probability=True, kernel="linear", C=2.8, gamma=.0073,verbose=10)

In [12]:
svc.fit(tr_st,tr_ft)

[LibSVM]

SVC(C=2.8, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=0.0073, kernel='linear',
  max_iter=-1, probability=True, random_state=None, shrinking=True,
  tol=0.001, verbose=10)

In [0]:
yhat_ts = svc.predict(val_st)

In [0]:
yhat_tr = svc.predict(Xtr)

In [0]:
acc = np.mean(yhat_tr == ytr)
print('Training Accuaracy = {0:f}'.format(acc))

In [15]:
acc = np.mean(yhat_ts == val_ft)
print('Accuaracy = {0:f}'.format(acc))

Accuaracy = 0.790541


In [16]:
from sklearn.metrics import classification_report
target_names = ['class 0', 'class 1']
print(classification_report(val_ft, yhat_ts, target_names=target_names))

              precision    recall  f1-score   support

     class 0       0.76      0.75      0.75       256
     class 1       0.81      0.82      0.82       336

   micro avg       0.79      0.79      0.79       592
   macro avg       0.79      0.79      0.79       592
weighted avg       0.79      0.79      0.79       592



In [0]:
print("Future:")
unique, counts = np.unique(np.concatenate((tr_ft,val_ft)), return_counts=True)
print("Total: ",dict(zip(unique, counts)))
unique, counts = np.unique(tr_ft, return_counts=True)
print("Train: ",dict(zip(unique, counts)))
unique, counts = np.unique(val_ft, return_counts=True)
print("Test: ",dict(zip(unique, counts)))


Future:
Total:  {0.0: 1012, 1.0: 1356}
Train:  {0.0: 756, 1.0: 1020}
Test:  {0.0: 256, 1.0: 336}


In [0]:
# Xtr, Xts, ytrp, ytsp = train_test_split(X, yp, test_size=0.3, random_state=42)


In [0]:
svcp = svm.SVC(probability=True, kernel="linear", C=2.8, gamma=.0073,verbose=10)

In [18]:
svcp.fit(tr_st,tr_pt)

[LibSVM]

SVC(C=2.8, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=0.0073, kernel='linear',
  max_iter=-1, probability=True, random_state=None, shrinking=True,
  tol=0.001, verbose=10)

In [0]:
yhat_tsp = svcp.predict(val_st)

In [0]:
yhat_trp = svcp.predict(Xtr)

In [0]:
acc = np.mean(yhat_trp == ytrp)
print('Training Accuaracy = {0:f}'.format(acc))

Training Accuaracy = 1.000000


In [20]:
acc = np.mean(yhat_tsp == val_pt)
print('Accuaracy = {0:f}'.format(acc))

Accuaracy = 0.792230


In [21]:
target_names = ['class 0', 'class 1']
print(classification_report(val_pt, yhat_tsp, target_names=target_names))

              precision    recall  f1-score   support

     class 0       0.78      0.90      0.84       352
     class 1       0.81      0.63      0.71       240

   micro avg       0.79      0.79      0.79       592
   macro avg       0.80      0.77      0.77       592
weighted avg       0.79      0.79      0.79       592



In [0]:
print("Past:")
unique, counts = np.unique(np.concatenate((tr_pt,val_pt)), return_counts=True)
print("Total: ",dict(zip(unique, counts)))
unique, counts = np.unique(tr_pt, return_counts=True)
print("Train: ",dict(zip(unique, counts)))
unique, counts = np.unique(val_pt, return_counts=True)
print("Test: ",dict(zip(unique, counts)))


Past:
Total:  {0.0: 1452, 1.0: 916}
Train:  {0.0: 1100, 1.0: 676}
Test:  {0.0: 352, 1.0: 240}


In [0]:
C_test = [0.1,1,2.5,5,10]
gam_test = [0.001,0.01,0.1]

nC = len(C_test)
ngam = len(gam_test)
acc = np.zeros((nC,ngam))

In [0]:
# Xtr, Xts, ytr, yts = train_test_split(X, yf, test_size=0.3, random_state=42)

# TODO:  Print the accuracy matrix
for index_C , C in enumerate(C_test):
  for index_G , gamma in enumerate(gam_test):
    svc2 = svm.SVC(probability=True, kernel="linear", C=C, gamma=gamma, verbose=10)
    svc2.fit(tr_st,tr_ft)
    yhat_testing_cross = svc2.predict(val_st)
    acc[index_C , index_G ] = np.mean(yhat_testing_cross == val_ft)
    print(C,gamma)
    print('Acc', acc[index_C,index_G])

[LibSVM]0.1 0.001
Acc 0.7922297297297297
[LibSVM]0.1 0.01
Acc 0.7922297297297297
[LibSVM]0.1 0.1
Acc 0.7922297297297297
[LibSVM]1 0.001
Acc 0.7719594594594594
[LibSVM]1 0.01
Acc 0.7719594594594594
[LibSVM]1 0.1
Acc 0.7719594594594594
[LibSVM]2.5 0.001
Acc 0.7719594594594594
[LibSVM]2.5 0.01
Acc 0.7719594594594594
[LibSVM]2.5 0.1
Acc 0.7719594594594594
[LibSVM]5 0.001
Acc 0.7719594594594594
[LibSVM]5 0.01
Acc 0.7719594594594594
[LibSVM]5 0.1
Acc 0.7719594594594594
[LibSVM]10 0.001
Acc 0.7719594594594594
[LibSVM]10 0.01
Acc 0.7719594594594594
[LibSVM]10 0.1
Acc 0.7719594594594594


In [0]:
optimal_index_C , optimal_index_Gamma = np.unravel_index(acc.argmax(),acc.shape)

print("Maximum Accuracy",acc[optimal_index_C][optimal_index_Gamma])
opt_c_f = C_test[optimal_index_C]
opt_g_f = gam_test[optimal_index_Gamma]
print("Optimal C:", C_test[optimal_index_C])
print('Optimal Gamma:' , gam_test[optimal_index_Gamma])

Maximum Accuracy 0.7922297297297297
Optimal C: 0.1
Optimal Gamma: 0.001


In [22]:
testfile='test.txt'
ts_st,ts_ft,ts_pt=load_test(testfile)

#preprocessing
ts_st = preprocess_sentences(ts_st)

#feature extraction
ts_st = sent_to_features(ts_st)



593


In [0]:

ts_st = vectorizer.transform(ts_st)

In [0]:
ts_st = tfidfconverter.transform(ts_st)


In [0]:
tr_val_st = vectorizer.transform(documents)

In [0]:
tr_val_st = tfidfconverter.transform(documents)

In [25]:

svc2 = svm.SVC(probability=True, kernel="linear", C=0.1, gamma=0.001, verbose=10)
print(tr_val_st.shape,tr_ft.shape,val_ft.shape)
svc2.fit(tr_val_st,np.concatenate((tr_ft,val_ft)))


(2368, 39471) (1776,) (592,)
[LibSVM]

SVC(C=0.1, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=0.001, kernel='linear',
  max_iter=-1, probability=True, random_state=None, shrinking=True,
  tol=0.001, verbose=10)

In [26]:
yhat_testing = svc2.predict(ts_st)
accu = np.mean(yhat_testing == ts_ft)
print('Test Accuracy: ',accu)

Test Accuracy:  0.5598650927487352


In [27]:
target_names = ['class 0', 'class 1']
print(classification_report(ts_ft, yhat_testing, target_names=target_names))

              precision    recall  f1-score   support

     class 0       0.97      0.11      0.20       293
     class 1       0.53      1.00      0.70       300

   micro avg       0.56      0.56      0.56       593
   macro avg       0.75      0.55      0.45       593
weighted avg       0.75      0.56      0.45       593



In [0]:
# Xtr, Xts, ytrp, ytsp = train_test_split(X, yp, test_size=0.3, random_state=42)
C_test = [0.1,1,2.5,5,10]
gam_test = [0.001,0.01,0.1]

nC = len(C_test)
ngam = len(gam_test)
acc = np.zeros((nC,ngam))
# TODO:  Print the accuracy matrix
for index_C , C in enumerate(C_test):
  for index_G , gamma in enumerate(gam_test):
    svc2 = svm.SVC(probability=True, kernel="linear", C=C, gamma=gamma, verbose=10)
    svc2.fit(tr_st,tr_pt)
    yhat_testing_cross = svc2.predict(val_st)
    acc[index_C , index_G] = np.mean(yhat_testing_cross == val_pt)
    print(C,gamma)
    print('Acc', acc[index_C,index_G])

[LibSVM]0.1 0.001
Acc 0.7871621621621622
[LibSVM]0.1 0.01
Acc 0.7871621621621622
[LibSVM]0.1 0.1
Acc 0.7871621621621622
[LibSVM]1 0.001
Acc 0.785472972972973
[LibSVM]1 0.01
Acc 0.785472972972973
[LibSVM]1 0.1
Acc 0.785472972972973
[LibSVM]2.5 0.001
Acc 0.785472972972973
[LibSVM]2.5 0.01
Acc 0.785472972972973
[LibSVM]2.5 0.1
Acc 0.785472972972973
[LibSVM]5 0.001
Acc 0.785472972972973
[LibSVM]5 0.01
Acc 0.785472972972973
[LibSVM]5 0.1
Acc 0.785472972972973
[LibSVM]10 0.001
Acc 0.785472972972973
[LibSVM]10 0.01
Acc 0.785472972972973
[LibSVM]10 0.1
Acc 0.785472972972973


In [0]:
optimal_index_C , optimal_index_Gamma = np.unravel_index(acc.argmax(),acc.shape)

print("Maximum Accuracy",acc[optimal_index_C][optimal_index_Gamma])
opt_c_p = C_test[optimal_index_C]
opt_g_p = gam_test[optimal_index_Gamma]
print("Optimal C:", C_test[optimal_index_C])
print('Optimal Gamma:' , gam_test[optimal_index_Gamma])

Maximum Accuracy 0.7871621621621622
Optimal C: 0.1
Optimal Gamma: 0.001


In [28]:
svc2 = svm.SVC(probability=True, kernel="linear", C=0.1, gamma=0.001, verbose=10)
svc2.fit(tr_val_st,np.concatenate((tr_pt,val_pt)))

[LibSVM]

SVC(C=0.1, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=0.001, kernel='linear',
  max_iter=-1, probability=True, random_state=None, shrinking=True,
  tol=0.001, verbose=10)

In [29]:
yhat_testing = svc2.predict(ts_st)
acc = np.mean(yhat_testing == ts_pt)
print('Test Accuracy: ',acc)
target_names = ['class 0', 'class 1']
print(classification_report(ts_pt, yhat_testing, target_names=target_names))

Test Accuracy:  0.6003372681281619
              precision    recall  f1-score   support

     class 0       0.58      1.00      0.73       324
     class 1       0.97      0.12      0.22       269

   micro avg       0.60      0.60      0.60       593
   macro avg       0.77      0.56      0.47       593
weighted avg       0.76      0.60      0.50       593

